In [ ]:
# Importing dependencies
from sklearn.datasets.samples_generator import make_blobs
from matplotlib import pyplot
import numpy as np
from numpy import where
from collections import Counter
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

sns.set_style("whitegrid", {'axes.grid' : False})

from keras.layers import Dense, Dropout, LeakyReLU

from keras.models import Sequential
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils import to_categorical
import itertools

%cd C:\\Users\\Ben\\Desktop\\UNH Classes\\Data 903

In [ ]:
# Make data
from  sklearn.datasets import make_classification as makeclass
X,y = makeclass(n_samples=10000, n_features=15, n_informative=10, n_redundant=0, 
          n_repeated=0, n_classes=6, n_clusters_per_class=3, weights=None, flip_y=0.01, 
          class_sep=0.7, hypercube=False, shift=2, scale=1.0, shuffle=True, random_state=682502)

y = to_categorical(y)

# split into train and test
n_train = 5000
trainX, testX = X[:n_train, :], X[n_train:, :]
trainy, testy = y[:n_train], y[n_train:]


In [ ]:
trainX.shape

In [ ]:
##### Part 1 for Sigmoid and ReLU #####
#######################################

# Number of epochs (only altered while testing)
EP = 100

# Set hyperparamaters for non-leaky activations
act = ['sigmoid', 'relu']
nod = [10, 20, 50]
bat = [8, 16, 32]
lay = [1,2]

#function for permutations
def hyper_paramters(act, nod, bat, lay):
    
    lists = [act, nod, bat, lay]
    
    perm = list(itertools.product(*lists))
    
    return perm

#execution of function
perm = hyper_paramters(act,nod,bat,lay)


#length of all combinations
len(perm)


In [ ]:
# Sigmoid and ReLU model function
def RunNN(act,nod,bat,lay):
    print("")
    print("Working on model configuration with activation: " + str(act) + ", nodes: " + str(nod)+ ", batches: " + str(bat) +
          ", and layers: " + str(lay))
    print(str(len(perm)-i) + " configurations left to try.")
    print("")
    
    model = Sequential()
    model.add(Dense(nod, input_dim=15, activation=act))
    
    model.add(Dense(nod, activation=act))
    
    if lay == 2:
        model.add(Dense(nod, activation=act))
        
        model.add(Dense(6, activation='softmax'))
    


        # compile model
        opt = RMSprop(lr=0.01, rho=0.9)
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

        # fit model
        history[i]= model.fit(trainX, trainy, validation_data=(testX, testy), epochs=EP, verbose=1, batch_size=bat)


        configs.append(str(act) + ", " + str(nod) + ", " + str(bat) + ", "+ str(lay))    
        
              
    else:
    
        model.add(Dense(6, activation='softmax'))



        # compile model
        opt = RMSprop(lr=0.01, rho=0.9)
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

        # fit model
        history[i]= model.fit(trainX, trainy, validation_data=(testX, testy), epochs=EP, verbose=1, batch_size=bat)


        configs.append(str(act) + ", " + str(nod) + ", " + str(bat) + ", "+ str(lay))

In [ ]:
# create objects for storing configurations and model performance
configs = []
history = {}

# Run the sigmoid/relu model by entering the combinations through the model function
for i in range(0, len(perm)):
    RunNN(perm[i][0],perm[i][1],perm[i][2],perm[i][3])
    

In [ ]:
### Transform Sigmoid / ReLU results and save output ###
########################################################

# Create rows by repeating the configuration settings to match the number of epochs

CF = pd.DataFrame(configs)
newCF = pd.DataFrame(np.repeat(CF.values,EP,axis=0))
newCF.columns = ['config']
newCF = newCF['config'].str.split(',', 3, expand=True)
newCF.columns = ['act','nod','bat','lay']
newCF

# Loop through history dict to create a DF with epoch number and acc score
DF = []
for i in range(0, len(history)):
    epoch_res = pd.DataFrame(history[i].history)
    epoch_res['epoch'] = epoch_res.index+1
    epoch_res = epoch_res[['epoch','acc']]
    DF.append(epoch_res)

# Combine epoch configurations with their respective epochs and acc scores 

DF2 = pd.concat(DF).reset_index(drop=True)
DF3 = DF2[['epoch', 'acc']]
DF4 = pd.merge(newCF,DF3, left_index=True, right_index=True)

DF5 = pd.pivot_table(DF4, columns='epoch', values='acc',index =['act','nod','bat','lay'])

DF5.to_csv('nonleaky.csv')
DF5.to_excel('nonleaky.xlsx')
DF5

In [ ]:
##### Part 1 for Leaky ReLU ###########
#######################################

# Number of epochs (only altered while testing)
EP = 100

# Set hyperparamaters for leaky activation
# act = ['sigmoid', 'relu']
nod = [10, 20, 50]
bat = [8, 16, 32]
lay = [1,2]

#function for permutations
def hyper_leaky(nod, bat, lay):
    
    lists = [nod, bat, lay]
    
    perm = list(itertools.product(*lists))
    
    return perm

#execution of function
perm = hyper_leaky(nod,bat,lay)


#length of all combinations
len(perm)




In [ ]:
# Leaky ReLU model function
def RunNNLeaky(nod,bat,lay):
    print("")
    print("Working on model configuration with nodes: " + str(nod)+ ", batches: " + str(bat) + ", and layers: " + str(lay))
    print(str(len(perm)-i) + " configurations left to try.")
    print("")
    
    model = Sequential()
    model.add(Dense(nod, input_dim=15))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(nod))
    model.add(LeakyReLU(alpha=0.1))
    if lay == 2:
        model.add(Dense(nod))
        model.add(LeakyReLU(alpha=0.1))
        
        model.add(Dense(6, activation='softmax'))
    


        # compile model
        opt = RMSprop(lr=0.01, rho=0.9)
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

        # fit model
        history[i]= model.fit(trainX, trainy, validation_data=(testX, testy), epochs=EP, verbose=1, batch_size=bat)


        configs.append(str(nod) + ", " + str(bat) + ", "+ str(lay))    
        
              
    else:
    
        model.add(Dense(6, activation='softmax'))



        # compile model
        opt = RMSprop(lr=0.01, rho=0.9)
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

        # fit model
        history[i]= model.fit(trainX, trainy, validation_data=(testX, testy), epochs=EP, verbose=1, batch_size=bat)


        configs.append(str(nod) + ", " + str(bat) + ", "+ str(lay))



In [ ]:
# create objects for storing configurations and model performance
configs = []
history = {}

# Run the model by entering the combinations through the model function
for i in range(0, len(perm)):
    RunNNLeaky(perm[i][0],perm[i][1],perm[i][2])

In [ ]:
### Transform Leaky ReLU results and save output ###
####################################################

# Create rows by repeating the configuration settings to match the number of epochs
CF = pd.DataFrame(configs)
newCF = pd.DataFrame(np.repeat(CF.values,EP,axis=0))
newCF.columns = ['config']
newCF = newCF['config'].str.split(',', 3, expand=True)
newCF.columns = ['nod','bat','lay']
newCF


# Loop through history dict to create a DF with epoch number and acc score
DF = []
for i in range(0, len(history)):
    epoch_res = pd.DataFrame(history[i].history)
    epoch_res['epoch'] = epoch_res.index+1
    epoch_res = epoch_res[['epoch','acc']]
    DF.append(epoch_res)

# Combine epoch configurations with their respective epochs and acc scores 

DF2 = pd.concat(DF).reset_index(drop=True)
DF3 = DF2[['epoch', 'acc']]
DF4 = pd.merge(newCF,DF3, left_index=True, right_index=True)
DF4['act'] = 'leaky'
DF4 = DF4[['act', 'nod', 'bat', 'lay', 'epoch', 'acc']]

DF5 = pd.pivot_table(DF4, columns='epoch', values='acc',index =['act','nod','bat','lay'])

# DF5.to_csv('leaky.csv')
# DF5.to_excel('leaky.xlsx')

DF5

In [ ]:
##### Part 2 ##########################
#######################################

# Number of epochs (only altered while testing)
EP = 100

# Set hyperparamaters for leaky activation

opt = ['sgd', 'rmsprop', 'adam']
wgt = ['glorot_uniform', 'glorot_normal','he_uniform', 'he_normal']
drp = [0.2, 0.4, 0.5]

#function for permutations
def hyper_2(opt, wgt, drp):
    
    lists = [opt, wgt, drp]
    
    perm = list(itertools.product(*lists))
    
    return perm

#execution of function
perm = hyper_2(opt, wgt, drp)


#length of all combinations
len(perm)



In [ ]:
def RunNN2(opt,wgt,drp):
    print("")
    print("Working on model configuration with optimization: " + str(opt)+ ", weights: " + str(wgt) + ", and dropout: " + str(drp))
    print(str(len(perm)-i) + " configurations left to try.")
    print("")
    
    model = Sequential()
    model.add(Dense(50, input_dim=15, activation='relu'))
    
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(drp))
    model.add(Dense(6, activation='softmax'))



    # compile model
    if opt == 'sgd':
        optz = SGD(lr=0.01)
        model.compile(loss='categorical_crossentropy', optimizer=optz, metrics=['accuracy'])

        # fit model
        history[i]= model.fit(trainX, trainy, validation_data=(testX, testy), epochs=EP, verbose=1, batch_size=32)


        configs.append(str(opt) + ", " + str(wgt) + ", "+ str(drp))
        
        
    elif opt == 'rmsprop':
        optz = RMSprop(lr=0.01, rho=0.9)
        model.compile(loss='categorical_crossentropy', optimizer=optz, metrics=['accuracy'])

        # fit model
        history[i]= model.fit(trainX, trainy, validation_data=(testX, testy), epochs=EP, verbose=1, batch_size=32)


        configs.append(str(opt) + ", " + str(wgt) + ", "+ str(drp))    

    else:
        optz = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
        model.compile(loss='categorical_crossentropy', optimizer=optz, metrics=['accuracy'])

        # fit model
        history[i]= model.fit(trainX, trainy, validation_data=(testX, testy), epochs=EP, verbose=1, batch_size=32)


        configs.append(str(opt) + ", " + str(wgt) + ", "+ str(drp))    



In [ ]:
# create objects for storing configurations and model performance
configs = []
history = {}

# Run the model by entering the combinations through the model function
for i in range(0, len(perm)):
    RunNN2(perm[i][0],perm[i][1],perm[i][2])

In [ ]:

# Create rows by repeating the configuration settings to match the number of epochs
CF = pd.DataFrame(configs)
newCF = pd.DataFrame(np.repeat(CF.values,EP,axis=0))
newCF.columns = ['config']
newCF = newCF['config'].str.split(',', 3, expand=True)
newCF.columns = ['opt','wgt','drp']
newCF


# Loop through history dict to create a DF with epoch number and acc score
DF = []
for i in range(0, len(history)):
    epoch_res = pd.DataFrame(history[i].history)
    epoch_res['epoch'] = epoch_res.index+1
    epoch_res = epoch_res[['epoch','acc']]
    DF.append(epoch_res)

# Combine epoch configurations with their respective epochs and acc scores 

DF2 = pd.concat(DF).reset_index(drop=True)
DF3 = DF2[['epoch', 'acc']]
DF4 = pd.merge(newCF,DF3, left_index=True, right_index=True)
DF4['act'] = 'leaky'
DF4 = DF4[['opt','wgt','drp', 'epoch', 'acc']]

DF5 = pd.pivot_table(DF4, columns='epoch', values='acc',index =['opt','wgt','drp'])

DF5.to_csv('part2.csv')
DF5.to_excel('part2.xlsx')

DF5